In [1]:
from recom.datasets import load_ml_small_rating

# load data
# not that I use leave-one-out method to construct the testing set, where
# the latest rated item is masked and added to the testing set as an evaluation.
dataset = load_ml_small_rating(need_raw=True, time_ord=True, test_perc=0.1)

# load features
ratings = dataset['raw']    
ratings_train_dict = dataset['train_dict']
ratings_test_dict = dataset['test_dict']
n_user = dataset['n_user']
n_item = dataset['n_item']
user2ix = dataset['user2ix']
ix2user = dataset['ix2user']
item2ix = dataset['item2ix']
ix2item = dataset['ix2item']

del dataset

print(f'Users: {n_user}, Items: {n_item}. Sparsity: {round(1-len(ratings)/n_user/n_item, 4)}')
print(f'User reduced from {len(user2ix.keys())} to {len(ratings_train_dict.keys())}')

Users: 609, Items: 9562. Sparsity: 0.983
User reduced from 609 to 608


In [61]:
import torch.nn as nn
from torch import Tensor, LongTensor, sigmoid
from torch.nn.functional import logsigmoid


class GMF(nn.Module):
    """ General Matrix Factorization """
    def __init__(self, n_user, n_item, k_dim, activation='sigmoid') -> None:
        assert activation in ['sigmoid', 'identity'], f'Invalid activation function for GMF {activation}.'

        super(GMF, self).__init__()
        # embeddings
        self.embedding_user = nn.Embedding(n_user, k_dim)
        self.embedding_item = nn.Embedding(n_item, k_dim)
        # weights
        self.linear = nn.Linear(k_dim, 1)
        # activation
        self.activation = sigmoid if activation=='sigmoid' else nn.Identity()
        # init param
        nn.init.normal_(self.embedding_user.weight, mean=0, std=1)
        nn.init.normal_(self.embedding_item.weight, mean=0, std=1)

    def forward(self, user, item):
        user_emb = self.embedding_user(user)
        item_emb = self.embedding_item(item)

        return self.activation(
            self.linear(user_emb*item_emb)
        )


class MLP(nn.Module):
    """ Multi-Layer Perceptron """
    def __init__(self, n_user, n_item, k_dim 
                     , num_layers, p_dropout) -> None:
        
        super(GMF, self).__init__()
        # embeddings
        self.embedding_user = nn.Embedding(n_user, k_dim)
        self.embedding_item = nn.Embedding(n_item, k_dim)



class NeuMF(nn.Module):
    """ Neural Matrix Factorization: Fusion of GMF and MLP """
    pass


gmf = GMF(10, 20, 10, activation='sigmoid')

users = LongTensor([0,1,2,3,4])
items = LongTensor([1,1,1,3,4])

gmf(users, items)

tensor([[0.3903],
        [0.4543],
        [0.3995],
        [0.4731],
        [0.5191]], grad_fn=<SigmoidBackward0>)

In [49]:
gmf.embedding_user(users)

tensor([[ 0.0914, -0.0433, -0.0532, -0.0686,  0.0317,  0.0350, -0.0755, -0.0882,
          0.1961, -0.0904],
        [-0.0891, -0.0040,  0.0249,  0.0659,  0.0359,  0.0736, -0.0962,  0.1316,
          0.1331,  0.0945],
        [-0.2859,  0.0470,  0.0252,  0.0615,  0.1043, -0.1550, -0.0327,  0.1001,
         -0.0188, -0.0043],
        [-0.0118,  0.1401, -0.0508,  0.2316,  0.0492,  0.0949, -0.0806,  0.1010,
          0.1429, -0.0956],
        [-0.0451, -0.0537,  0.0810, -0.0042,  0.0039, -0.0303, -0.0387, -0.0049,
          0.0189, -0.0301]], grad_fn=<EmbeddingBackward0>)

In [51]:
gmf.embedding_item(items)

tensor([[-0.0420,  0.1218, -0.1399, -0.2115,  0.1238, -0.0234,  0.0315,  0.1099,
          0.1311,  0.0386],
        [-0.0420,  0.1218, -0.1399, -0.2115,  0.1238, -0.0234,  0.0315,  0.1099,
          0.1311,  0.0386],
        [-0.0420,  0.1218, -0.1399, -0.2115,  0.1238, -0.0234,  0.0315,  0.1099,
          0.1311,  0.0386],
        [ 0.0295,  0.0525,  0.0866, -0.1791,  0.0441, -0.0125, -0.0089, -0.0674,
         -0.2411,  0.0215],
        [ 0.0689,  0.1505,  0.0462, -0.1141, -0.0659, -0.1473,  0.3071,  0.1171,
          0.1345,  0.1276]], grad_fn=<EmbeddingBackward0>)

In [53]:
ews = gmf.embedding_user(users)*gmf.embedding_item(items)
ews

tensor([[-0.0038, -0.0053,  0.0074,  0.0145,  0.0039, -0.0008, -0.0024, -0.0097,
          0.0257, -0.0035],
        [ 0.0037, -0.0005, -0.0035, -0.0139,  0.0044, -0.0017, -0.0030,  0.0145,
          0.0175,  0.0036],
        [ 0.0120,  0.0057, -0.0035, -0.0130,  0.0129,  0.0036, -0.0010,  0.0110,
         -0.0025, -0.0002],
        [-0.0003,  0.0074, -0.0044, -0.0415,  0.0022, -0.0012,  0.0007, -0.0068,
         -0.0344, -0.0021],
        [-0.0031, -0.0081,  0.0037,  0.0005, -0.0003,  0.0045, -0.0119, -0.0006,
          0.0025, -0.0038]], grad_fn=<MulBackward0>)

In [60]:
(gmf.linear.weight.data[0] * ews).sum(1)

tensor([ 0.0020,  0.0078,  0.0021,  0.0036, -0.0032], grad_fn=<SumBackward1>)

In [54]:
ews.sum(1)

tensor([ 0.0261,  0.0211,  0.0251, -0.0805, -0.0165], grad_fn=<SumBackward1>)

In [47]:
gmf.linear.bias

Parameter containing:
tensor([0.2394], requires_grad=True)

In [48]:
gmf.linear.weight

Parameter containing:
tensor([[-0.2281,  0.0944, -0.2378, -0.2669,  0.1129,  0.0884,  0.2206, -0.0763,
          0.2844,  0.1728]], requires_grad=True)